This notebook is for adding the final version of the corrected chlorophyll data to the binned, dark corrected, mission netCDF files. Rather than redo the correction for every mission, we're going to pull the relevant slice from the `quenchingCorrectionTests.nc` file, and add them to the appropriate mission nc file.

# Final quenching correction method: FLZn

The FLZn method performed the best of all the evaluated methods. The FLZn method has the following details:

1. Extrapolates chlfl between the last profile from the previous night and the first profile of the following night to use as the reference (non-quenched) chl data
2. Does not use a Zeu condition (hard limit or based on PAR)

# Initialisation

In [1]:
import xarray as xr
import os 
import re
import matplotlib.pyplot as plt

%matplotlib inline

## Finding the mission nc files

In [3]:
parentDir = '../../../data/mission-netcdf/darkCountCorrected/'
files = os.listdir(parentDir)
gFiles = [parentDir + ff for ff in files if os.path.isfile(parentDir+ff)]

## Importing the corrected chl data

In [4]:
filepath = '../../../data/quenching-evaluation-netcdf/quenchingCorrectionTests.nc'
cdata_all = xr.open_dataset(filepath)
cdata = cdata_all.sel(chlcorrmethod='FLZnSC').reset_coords('chlcorrmethod',drop=True)

## Making mission id reference dictionary

The glider missions are labeled differently within the `quenchingCorrectionTests` dataset, and in their filenames, so we need a reference dictionary to match the two together.

In [5]:
fullnames = [ff[:-3] for ff in files if os.path.isfile(parentDir+ff)]

In [6]:
id0 = [g[0] for g in fullnames]
id1 = [re.search('[0-9]*\.?[0-9]',gid).group() for gid in fullnames]
ids = [i0 + i1 for i0,i1 in zip(id0,id1)]
idref = {v: k for k, v in zip(ids,fullnames)}

In [7]:
idref

{'henry-mission18': 'h18',
 'grampus-mission1': 'g1',
 'grampus-mission2.1': 'g2.1',
 'grampus-mission2': 'g2',
 'grampus-mission3': 'g3',
 'grampus-mission4': 'g4',
 'grampus-mission5': 'g5',
 'grampus-mission6': 'g6',
 'grampus-mission7': 'g7',
 'grampus-mission8': 'g8',
 'henry-mission1': 'h1',
 'henry-mission2': 'h2',
 'henry-mission3': 'h3',
 'henry-mission4': 'h4',
 'henry-mission5': 'h5',
 'henry-mission6': 'h6',
 'henry-mission7.1': 'h7.1',
 'henry-mission7': 'h7',
 'henry-mission8': 'h8',
 'henry-mission9': 'h9',
 'henry-mission10': 'h10',
 'henry-mission11': 'h11',
 'henry-mission12': 'h12',
 'henry-mission13': 'h13',
 'henry-mission14': 'h14',
 'henry-mission15': 'h15',
 'henry-mission16': 'h16',
 'henry-mission17': 'h17'}

# Adding corrected data to mission files

In [8]:
newsaveparentdir = '../../../data/mission-netcdf/quenchingCorrected/'

for gf in gFiles:
    gdata = xr.open_dataset(gf)
    
    midlong = re.split('/',gf)[-1]
    mid = idref[midlong[:-3]]
    
    for dd in ['east','west']:
        try:
            chlcorr = cdata[mid+'_'+dd]
            gdata['chlflCorr_'+dd] = chlcorr
        except KeyError:
            continue
    
    gdata.to_netcdf(newsaveparentdir+midlong)
    